継続は色々と説明されているが、しかしそれらの説明は「正しく」はあるものの、使いこなせるというものでもない。そのような「正確な」定義はともかくとして、(そのような示唆に富み、かつ有益なドキュメントは各位先輩Lisperが色々書いてくれている)。実際のところは使ってみて会得するのが一番いいので、その手習いの記録である。

結局のところ、我々はパソコンのことを「知らない」が「使える」。たぶん、継続も一緒だ。私達は時間のことは全くわからないが、しかし「時計」は使えるのだ。Schemeも全くわからないが「書ける」のであって、そういうものはたくさん存在している。

# 一番簡単な例

In [ ]:
(begin (displayln (call/cc 
          (lambda (cc)
            (displayln "ラムダの中に入ったよ！")
            (cc "帰るよ！じゃあね！")
            (displayln "あれ、みんなどこ？"))))
        (displayln "ただいま！"))


このような使い方は[大域脱出](https://ja.wikibooks.org/wiki/Scheme/%E7%B6%99%E7%B6%9A%E3%81%AE%E7%A8%AE%E9%A1%9E%E3%81%A8%E5%88%A9%E7%94%A8%E4%BE%8B)として説明されている。いわば **cc** を呼び出された瞬間に **call/cc** のところへ戻り、**cc** で受け取った値を引き渡しているということができる。もっとシンプルな大域脱出になると次のようになる。

In [ ]:
(require racket/function)

(define (f return) (+ (return 3) 2))
(displayln (f identity)) ;; (define (identity x) x)
(displayln (call/cc f))

このように考えれば、`call/cc`とは、そのポイントに帰ってくるような関数を渡す関数という風に考えることができる。

# ちょっとだけ簡単な例

「そのポイントに帰ってくるような関数を渡す関数」というわけだから、単純にループだって作ることもできるだろう。例えば、階乗を計算する`fact`は継続にすると次のように書ける。

In [ ]:
(require racket/function)

(define (fact/cc x)
  (define result 1)
  (define i 1)
  (let ([c (call/cc identity)])
    (set! result (* i result))
    (set! i (add1 i))
    (if (>= i x) result (c c))))

(fact/cc 10)

この手の線形的な例を考えるならば、実際は以下のような単純な再帰だっていい。

In [ ]:
(define (fact/recur x)
  (define (inner i result)
    (if (>= i x) result
        (inner (add1 i) (* result i))))
  (inner 1 1))

(fact/recur 10)

# ちょっとだけ複雑な例

「あるリストの要素を幾つも使って、何かしらの数を作る」という問題について考えてみよう。このとき`(list 2 3)`を使って`8`を取り出したい場合には次のような組み合わせが考えられる。

* (list 3 2) -> (3 3 2), (2 2 2 2)


In [21]:
(define (find-list-sum find-sum lst)
  (struct cc (proc use-elems))
  (let ([result (list)] [task (list)])
    ;-Task取り出し部分-------------------------------
    (define (task-next!)
      (let ([next (car task)])
           (set! task (cdr task))
           ((cc-proc next) (cc (cc-proc next) (cc-use-elems next)))))
    ;-再帰関数---------------------------------------
    (define (inner work use-elems)
      (let* ([c (call/cc (lambda (proc) (cc proc use-elems)))]
             [use-elems (cc-use-elems c)])
      (cond
       [(and (null? work) (null? use-elems)) result]
       [(null? use-elems) (task-next!)]
       [else 
         ;;- Taskの追加--
         (set! task (append (list (cc (cc-proc c) (cdr use-elems))) task))
         (cond
           [(and (not (null? use-elems)) (< (apply + work) find-sum)) (inner (cons (car use-elems) work) use-elems)]
           [(= (apply + work) find-sum) (set! result (append result (list work))) (task-next!)]
           [else (task-next!)])])))
    ;-終わり---------------------
    (inner (list) (sort lst >))))

(check-equal? (find-list-sum 1 (list)) (list))
(check-equal? (find-list-sum 10 (list 7 9 12)) (list))
(check-equal? (find-list-sum 8 (list 3 2)) (list (list 2 3 3) (list 2 2 2 2)))
(find-list-sum 10 (list 2 3 5 7))

'(#0=(3 7) #0# #1=(5 . #2=(5)) #1# #1# (2 3 . #2#) (2 2 3 3) (2 2 2 2 2))

再帰関数と比較した場合、「再帰関数」においては「値が更新されうるような変数」と「値が不変であるような変数」に分け、「値が更新されうるような変数」に関しては局所関数の引数で持つと、基本的には見通しが良くなる（ように個人的には思う）。「継続」の場合、もう少し状況が複雑である。

* 値が更新されうる変数
  * その値に戻ってくるような変数
  * 他の値に入れ替えるべき変数
* 値が更新されない変数

変更がかかるような変数であっても「セーブポイントとしてそこに戻るような値」と「別の値に入れ替える値」というのが二つ存在している。

今回の場合であるならば、「セーブポイントとして戻る値」というのは、「候補として選された値のリスト（上記でいうならば`work`の部分）」であり、そして別の値に入れ替える値というのは「値として選択された部分(`use-elems`)」である。かっこよく言ってしまえば **「ありえたもう一つの未来」** を保存しておくわけだ。

# もう一つちょっとだけ複雑な例

ユニークな要素で出来ているリストが与えられる。その要素の中から好きな数を選んで部分集合を作ったとき、いったい幾つの部分集合が出来るか？

In [32]:
(define (subsets lst)
  (let ([result (list)] [task (list)])
    
    (struct cc (proc rest))
    
    (define (task-next!)
      (let ([work (car task)])
        (set! task (cdr task))
        ((cc-proc work) (cc (cc-proc work) (cc-rest work)))))

    (define (inner work lst)
      (let* ([c (call/cc (lambda (c) (cc c lst)))]
             [rest (cc-rest c)])
        (cond
          [(and (null? work) (null? rest)) result]
          [(null? rest) (set! result (append result (list work))) (task-next!)]
          [else (set! task (append task (list (cc (cc-proc c) (cdr rest)))))
                (inner (append work (list (car rest))) (cdr rest))])))
    (inner (list) lst)))
      
;;--rackunit--
(require rackunit)
(check-equal? (subsets (list 1 2)) (list (list 1 2) (list 2) (list 1)))
(check-equal? (subsets (list 1 2 3))
                       (list (list 1 2 3)
                            (list 2 3) (list 1 3) (list 1 2)
                             (list 3) (list 2) (list 1)))

率直に言うと、継続の難しさというのはその概念が難易度が高いと言うよりも **「状態を保存して何が嬉しいの？」** というところに尽きるのではないかと思う。

全てのありうるべき **状態**を渡すときはいわゆる「再帰関数」になるのだけれど、全ての**状態**を保存するのは非常にコストがかかるというときには、継続でパックして、**値を変えたいところ** だけを渡してあげればよい。